In [1]:
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from datetime import datetime

In [2]:
def fetch_patient_info():
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password=input('What is your password?: '),
        database="mock_patient_project"
    )
    # print("Connected to MySQL Database")

    # Create cursor
    cursor = connection.cursor()

    try:
        # Query random medical record number from patient_data table
        query_medical_record_data = "SELECT * FROM patient_data ORDER BY RAND() LIMIT 1"
        cursor.execute(query_medical_record_data)
        medical_record_data = cursor.fetchone()

        # Get column names from cursor description
        column_names = [description[0] for description in cursor.description]

        # Construct DataFrame with appropriate column labels
        patient_info_df = pd.DataFrame([medical_record_data], columns=column_names)

        # Print the DataFrame
        # print("Patient Info DataFrame:")
        # print(patient_info_df)

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()
        # print("MySQL connection is closed")

    return patient_info_df

In [3]:
def generate_percentage():
    # Generate a random number from a normal distribution centered around 0.15
    # with a standard deviation that controls the spread of the bell curve
    mean = 0.015
    std_dev = 0.005  # Adjust this value to control the spread of the bell curve
    percentage = np.random.normal(mean, std_dev)
    
    # Ensure the generated percentage falls within the desired range [0.1, 0.2]
    percentage = max(0.01, min(0.02, percentage))
    
    return percentage

# Call the function to generate the percentage
percentage_to_fetch = generate_percentage()

print("Generated Percentage:", percentage_to_fetch)


Generated Percentage: 0.02


In [4]:
def determine_reason_for_encounter(age, gender):
    if age <= 18:
        reasons = [
            "Infectious diseases: This includes ear infections, respiratory infections (like pneumonia or RSV), and stomach flu. These are more common in younger children.",
            "Accidents and injuries: Falls, fractures, and cuts are frequent reasons for emergency room visits, especially in toddlers and school-aged children.",
            "Childhood illnesses: Asthma, allergies, and chronic conditions like diabetes may require hospitalization for management or flare-ups.",
            "Appendicitis: This is a more common concern for children over 5."
        ]
    elif age <= 64:
        reasons = [
            "Pregnancy and childbirth: Delivering a baby is a primary reason for hospitalization for women in this age group." if age < 35 and gender == "Female" and random.random() < (35 - age) / 35 else None,
            "Accidents and injuries: Car accidents, work-related injuries, and sports injuries are common.",
            "Chronic illnesses: Management of conditions like heart disease, diabetes, and cancer may involve hospital stays.",
            "Mental health concerns: Mental health episodes like depression or anxiety may require hospitalization for stabilization.",
            "Surgery: Procedures for appendicitis, ACL reconstruction, or other conditions may necessitate a hospital stay."
        ]
    else:
        reasons = [
            "Chronic illnesses: Exacerbations of heart disease, chronic obstructive pulmonary disease (COPD), and diabetes are frequent reasons for admission.",
            "Falls and fractures: These become more common as mobility declines.",
            "Pneumonia: This respiratory infection can be more serious for older adults.",
            "Complications from surgery: Recovery from surgery may take longer and require hospitalization for older adults."
        ]

    # Filter out None values and select a random reason
    reasons = [reason for reason in reasons if reason is not None]
    selected_reason = random.choice(reasons) if reasons else "Reason not specified"

    return selected_reason


In [ ]:
# Set the percentage of records to fetch
percentage_to_fetch = generate_percentage()  # Change this percentage as needed
num_iterations = 10  # Number of iterations
print("Generated Percentage:", percentage_to_fetch)

# Establish connection to MySQL Database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password=input('What is your password?: '),
    database="mock_patient_project"
)
cursor = connection.cursor()

# Get total number of records in patient_data table
cursor.execute("SELECT COUNT(*) FROM patient_data")
total_records = cursor.fetchone()[0]
print("Total Records in Patient Data Table:", total_records)

# Close cursor and connection
cursor.close()
connection.close()

# Calculate the number of records to fetch based on the percentage
num_records_to_fetch = int(total_records * (percentage_to_fetch / 100))

# Create empty list to store patient info dataframes
patient_info_dfs = pd.DataFrame()

# Iterate over the number of iterations
for i in range(num_iterations):
    # Call the function to fetch patient info
    patient_info_df = fetch_patient_info()

    # Determine reason for encounter based on age and gender
    age = patient_info_df['Age'].iloc[0]
    gender = patient_info_df['Gender'].iloc[0]
    reason_for_encounter = determine_reason_for_encounter(age, gender)

    # Add reason for encounter to DataFrame
    patient_info_df['ReasonForEncounter'] = reason_for_encounter

    # Add today's date to the DataFrame
    today_date = datetime.now().strftime("%Y-%m-%d")
    patient_info_df['EncounterDate'] = today_date

    # Append the returned dataframe to the list
    patient_info_dfs = pd.concat([patient_info_dfs, patient_info_df], ignore_index=True)

# Print the combined DataFrame
print(patient_info_dfs)

In [5]:
# # Function to generate encounters from past dates
# def generate_past_encounters(num_encounters):
#     encounters = []
#     today = datetime.now().date()
#     for _ in range(num_encounters):
#         # Generate a random date within the past 5 years
#         encounter_date = today - timedelta(days=random.randint(1, 1825))  # 5 years * 365 days = 1825 days
#         encounters.append(encounter_date)
#     return encounters

# # Set the percentage of records to fetch
# percentage_to_fetch = generate_percentage()  # Change this percentage as needed
# num_iterations = 10  # Number of iterations
# num_records_to_fetch = 3500  # Number of encounters from past dates
# print("Generated Percentage:", percentage_to_fetch)

# # Generate past encounter dates
# past_encounters = generate_past_encounters(num_records_to_fetch)

# # Create empty list to store patient info dataframes
# patient_info_dfs = pd.DataFrame()

# # Iterate over the number of iterations
# for i in range(num_records_to_fetch):
#     # Print the current iteration number
#     print("Iteration:", i+1)  
    
#     # Randomly select a past encounter date
#     encounter_date = random.choice(past_encounters)

#     # Call the function to fetch patient info
#     patient_info_df = fetch_patient_info()

#     # Determine reason for encounter based on age and gender
#     age = patient_info_df['Age'].iloc[0]
#     gender = patient_info_df['Gender'].iloc[0]
#     reason_for_encounter = determine_reason_for_encounter(age, gender)

#     # Add reason for encounter and encounter date to DataFrame
#     patient_info_df['ReasonForEncounter'] = reason_for_encounter
#     patient_info_df['EncounterDate'] = encounter_date

#     # Append the returned dataframe to the list
#     patient_info_dfs = pd.concat([patient_info_dfs, patient_info_df], ignore_index=True)

# # Print the combined DataFrame
# print(patient_info_dfs)


Generated Percentage: 0.015375025775929566
        FirstName  LastName              Name DateOfBirth  Age  Gender  \
0           Robin      Lowe        Robin Lowe  1969-01-26   59    Male   
1       Christian    Graham  Christian Graham  1935-05-15   43    Male   
2        Jonathan   Wiggins  Jonathan Wiggins  1964-05-20   65    Male   
3           Kevin     Cross       Kevin Cross  1975-04-23    3    Male   
4     Christopher      Hall  Christopher Hall  1958-05-16   46  Female   
...           ...       ...               ...         ...  ...     ...   
3495        April      Cruz        April Cruz  1957-08-04   49    Male   
3496     Jonathan      West     Jonathan West  1964-01-26   40  Female   
3497         Eric    Martin       Eric Martin  1963-06-18   26  Female   
3498        Peter  Peterson    Peter Peterson  1964-04-28   66    Male   
3499    Elizabeth    Murphy  Elizabeth Murphy  2001-08-29   33    Male   

     MedicalRecordNumber                                 ReasonForEn

In [7]:
def upload_to_mysql(dataframe, table_name):
    # Establish connection to MySQL Database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password=input('What is your password?: '),
        database="mock_patient_project"
    )
    cursor = connection.cursor()

    try:
        # Iterate over each row in the DataFrame and insert it into the MySQL table
        for index, row in dataframe.iterrows():
            # Prepare column names string
            columns_str = ', '.join(dataframe.columns)

            # Prepare placeholders for data values
            placeholders = ', '.join(['%s'] * len(row))

            # Construct SQL query for row insertion
            sql = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

            # Execute SQL query with row data
            cursor.execute(sql, tuple(row))

        # Commit changes
        connection.commit()

    except Exception as e:
        print("Error:", e)
        connection.rollback()

    finally:
        # Close cursor and connection
        cursor.close()
        connection.close()


In [8]:
upload_to_mysql(patient_info_dfs, 'encounters')